# Advanced Examples

In [1]:
%config InlineBackend.print_figure_kwargs = {'bbox_inches': 'tight', 'dpi': 110}
%load_ext autoreload
%autoreload 2
import logging, warnings
logging.getLogger("pymc").setLevel(logging.FATAL)
warnings.filterwarnings("ignore")

## PyMC

The [Example](example.html) page introduces how to use *muse-inference* for a problem defined with PyMC. Here we consider a more complex problem to highlight additional features. In particular:

* We can estimate any number of parameters with any shapes. Here we have a 2-dimensional array $\mu$ and a scalar $\theta$. Note that by default, *muse-inference* considers any variables which do not depend on others as "parameters" (i.e. the "leaves" of the probabilistic graph). However, the algorithm is not limited to such parameters, and any choice can be selected by providing a list of `params` to the `PyMCMuseProblem` constructor.

* We can work with distributions with limited domain support. For example, below we use the $\rm Beta$ distribution with support on $(0,1)$ and the $\rm LogNormal$ distribution with support on $(0,\infty)$. All necessary transformations are handled internally.

* The data and latent space can include any number of variables, with any shapes. Below we demonstrate an $x$ and $z$ which are 2-dimensional arrays. 

First, load the relevant packages:

In [2]:
%pylab inline
import pymc as pm
from muse_inference.pymc import PyMCMuseProblem

Populating the interactive namespace from numpy and matplotlib


Then define the problem,

In [3]:
def gen_funnel(x=None, σ=None, μ=None):
    with pm.Model() as model:
        μ = pm.Beta("μ", 2, 5, size=2) if μ is None else μ
        σ = pm.Normal("σ", 0, 3) if σ is None else σ
        z = pm.LogNormal("z", μ, np.exp(σ/2), size=(100, 2))
        x = pm.Normal("x", z, 1, observed=x)
    return model

generate the model and some data, given some chosen true values of parameters,

In [4]:
θ_true = dict(μ=[0.3, 0.7], σ=1)
with gen_funnel(**θ_true):
    x_obs = pm.sample_prior_predictive(1, random_seed=0).prior.x[0,0]
model = gen_funnel(x=x_obs)
prob = PyMCMuseProblem(model)

and finally, run MUSE:

In [5]:
θ_start = dict(μ=[0.5, 0.5], σ=0)
result = prob.solve(θ_start=θ_start, progress=True)

MUSE:   0%|          | 0/5050 [00:00<?, ?it/s]

MUSE:   0%|          | 7/5050 [00:00<01:15, 66.90it/s]

MUSE:   0%|          | 16/5050 [00:00<01:06, 75.70it/s]

MUSE:   1%|          | 27/5050 [00:00<00:57, 87.08it/s]

MUSE:   1%|          | 36/5050 [00:00<00:57, 87.29it/s]

...

get_H:  40%|████      | 4/10 [00:00<00:00, 12.42it/s]

get_H:  60%|██████    | 6/10 [00:00<00:00, 14.94it/s]

get_H:  80%|████████  | 8/10 [00:00<00:00, 15.02it/s]

get_H: 100%|██████████| 10/10 [00:00<00:00, 10.49it/s]

get_H: 100%|██████████| 10/10 [00:00<00:00, 11.51it/s]

When there are multiple parameters, the starting guess should be specified as as a dictionary, as above.

The parameter estimate is returned as a dictionary,

In [6]:
result.θ

{'μ': array([0.40489074, 0.42095688]), 'σ': array(0.91969963)}

 and the covariance as matrix, with parameters concatenated in the order they appear in the model (or in the order specified in `params`, if that was used):

In [7]:
result.Σ

array([[ 1.86115895e-02, -8.11185411e-05, -5.68335978e-03],
       [-8.11185411e-05,  3.68882367e-02, -1.71506153e-02],
       [-5.68335978e-03, -1.71506153e-02,  2.19031887e-02]])

The `result.ravel` and `result.unravel` functions can be used to convert between dictionary and vector representations of the parameters. For example, to compute the standard deviation for each parameter (the square root of the diagonal of the covariance):

In [8]:
result.unravel(np.sqrt(np.diag(result.Σ)))

{'μ': array([0.1364243 , 0.19206311]), 'σ': array(0.14799726)}

or to convert the mean parameters to a vector:

In [9]:
result.ravel(result.θ)

array([0.40489074, 0.42095688, 0.91969963])

## Jax

We can also use [Jax](https://jax.readthedocs.io/) to define the problem. In this case we will write out function to generate forward samples and to compute the posterior, and Jax will provide necessary gradients for free. To use Jax, load the necessary packages:

In [10]:
from functools import partial
import jax
import jax.numpy as jnp
from muse_inference.jax import JaxMuseProblem

Let's implement the noisy funnel problem from the [Example](example.html) page. To do so, extend `JaxMuseProblem` and define `sample_x_z`, `logLike`, and `logPrior`. 

In [11]:
class JaxFunnelMuseProblem(JaxMuseProblem):

    def __init__(self, N, **kwargs):
        super().__init__(**kwargs)
        self.N = N

    def sample_x_z(self, key, θ):
        keys = jax.random.split(key, 2)
        z = jax.random.normal(keys[0], (self.N,)) * jnp.exp(θ/2)
        x = z + jax.random.normal(keys[1], (self.N,))
        return (x, z)

    def logLike(self, x, z, θ):
        return -(jnp.sum((x - z)**2) + jnp.sum(z**2) / jnp.exp(θ) + 512*θ) / 2

    def logPrior(self, θ):
        return -θ**2 / (2*3**2)

Note that the super-class `JaxMuseProblem` will automatically take care of JIT compiling these functions, so you do *not* need to manually decorate them with `@jit`. However, if your functions contain code which cannot be JIT compiled, you should pass `super().__init__(jit=False)` to the super constructor in your `__init__` function.

The JAX MUSE interface also contains an option to use implicit differentation to compute the $H$ matrix (paper in prep). This is more numerically stable and faster than the default, which uses finite differences, although requires 2nd order automatic differentiation to work through your posterior. It's enabled by default, but can be disabled with `super().__init__(implicit_diff=False)`.

With the problem defined, we now generate some simulated data and save it to the problem with `set_x`. Note also the use of `PRNGKey` (rather than `RandomState` for PyMC/Numpy) for random number generation. 

In [12]:
prob = JaxFunnelMuseProblem(10000, implicit_diff=True)
key = jax.random.PRNGKey(0)
(x, z) = prob.sample_x_z(key, 0)
prob.set_x(x)

And finally, run MUSE:

In [13]:
prob.solve(θ_start=0., rng=jax.random.PRNGKey(1)) # warmup

MuseResult(-0.00396±0.0271)

In [14]:
result = prob.solve(θ_start=0., rng=jax.random.PRNGKey(1), progress=True)

MUSE:   0%|          | 0/5050 [00:00<?, ?it/s]

MUSE:   2%|▏         | 102/5050 [00:00<00:05, 863.26it/s]

MUSE:   4%|▍         | 203/5050 [00:00<00:06, 720.12it/s]

MUSE:   6%|▌         | 304/5050 [00:00<00:06, 749.23it/s]

MUSE:   8%|▊         | 397/5050 [00:00<00:05, 806.43it/s]

MUSE: 100%|██████████| 5050/5050 [00:00<00:00, 8374.80it/s]

get_H:   0%|          | 0/10 [00:00<?, ?it/s]

get_H: 100%|██████████| 10/10 [00:00<00:00, 684.37it/s]

Note that the solution here is obtained around 10X faster that the PyMC version of this in the [Example](example.html) page (the cloud machines which build these docs don't always achieve the 10X, but you see this if you run these examples locally). The Jax interface has much lower overhead, which will be noticeable for very fast posteriors like the one above. 

One convenient aspect of using Jax is that the parameters, `θ`, and latent space, `z`, can be any [pytree](https://jax.readthedocs.io/en/latest/pytrees.html), ie tuples, dictionaries, nested combinations of them, etc... (there is no requirement on the data format of the `x` variable). To demonstrate, consider a problem which is just two copies of the noisy funnel problem:

In [15]:
class JaxPyTreeFunnelMuseProblem(JaxMuseProblem):

    def __init__(self, N):
        super().__init__()
        self.N = N

    def sample_x_z(self, key, θ):
        (θ1, θ2) = (θ["θ1"], θ["θ2"])
        keys = jax.random.split(key, 4)
        z1 = jax.random.normal(keys[0], (self.N,)) * jnp.exp(θ1/2)
        z2 = jax.random.normal(keys[1], (self.N,)) * jnp.exp(θ2/2)        
        x1 = z1 + jax.random.normal(keys[2], (self.N,))
        x2 = z2 + jax.random.normal(keys[3], (self.N,))        
        return ({"x1":x1, "x2":x2}, {"z1":z1, "z2":z2})

    def logLike(self, x, z, θ):
        return (
            -(jnp.sum((x["x1"] - z["z1"])**2) + jnp.sum(z["z1"]**2) / jnp.exp(θ["θ1"]) + 512*θ["θ1"]) / 2
            -(jnp.sum((x["x2"] - z["z2"])**2) + jnp.sum(z["z2"]**2) / jnp.exp(θ["θ2"]) + 512*θ["θ2"]) / 2
        )

    def logPrior(self, θ):
        return - θ["θ1"]**2 / (2*3**2) - θ["θ2"]**2 / (2*3**2)

Here, `x`, `θ`, and `z` are all dictionaries. We generate the problem as usual, passing in parameters as dictionaries,

In [16]:
θ_true = dict(θ1=-1., θ2=2.)
θ_start = dict(θ1=0., θ2=0.)

In [17]:
prob = JaxPyTreeFunnelMuseProblem(10000)
key = jax.random.PRNGKey(0)
(x, z) = prob.sample_x_z(key, θ_true)
prob.set_x(x)

and run MUSE:

In [18]:
prob.solve(θ_start=θ_start, rng=jax.random.PRNGKey(0)) # warmup

MuseResult({θ1=-1±0.0582, θ2=2.03±0.0156})

In [19]:
result = prob.solve(θ_start=θ_start, rng=jax.random.PRNGKey(0), progress=True)

MUSE:   0%|          | 0/5050 [00:00<?, ?it/s]

MUSE:   2%|▏         | 102/5050 [00:00<00:11, 432.56it/s]

MUSE:   3%|▎         | 149/5050 [00:00<00:11, 444.86it/s]

MUSE:   4%|▍         | 197/5050 [00:00<00:10, 455.81it/s]

MUSE:   5%|▍         | 244/5050 [00:00<00:16, 285.31it/s]

...

MUSE:  52%|█████▏    | 2609/5050 [00:09<00:07, 311.60it/s]

MUSE: 100%|██████████| 5050/5050 [00:09<00:00, 5092.82it/s]

MUSE: 100%|██████████| 5050/5050 [00:09<00:00, 517.37it/s] 

get_H:   0%|          | 0/10 [00:00<?, ?it/s]

get_H: 100%|██████████| 10/10 [00:00<00:00, 193.54it/s]

The result is returned as a pytree:

In [20]:
result.θ

{'θ1': DeviceArray(-1.0020632, dtype=float32),
 'θ2': DeviceArray(2.027134, dtype=float32)}

and the covariance as a matrix:

In [21]:
result.Σ

array([[ 3.3819058e-03, -2.5075144e-05],
       [-2.5075142e-05,  2.4444878e-04]], dtype=float32)

The `result.ravel` and `result.unravel` functions can be used to convert between pytree and vector representations of the parameters. For example, to compute the standard deviation for each parameter (the square root of the diagonal of the covariance):

In [22]:
result.unravel(np.sqrt(np.diag(result.Σ)))

{'θ1': DeviceArray(0.05815415, dtype=float32),
 'θ2': DeviceArray(0.01563486, dtype=float32)}

or to convert the mean parameters to a vector:

In [23]:
result.ravel(result.θ)

DeviceArray([-1.0020632,  2.027134 ], dtype=float32)